In [ ]:
!pip install transformers


In [ ]:
!pip install sentence-transformers faiss-cpu 

In [ ]:
!pip install -U sentence-transformers

In [ ]:
!pip install faiss-cpu 
!pip install --upgrade transformers

In [ ]:
import faiss
import pandas as pd
from sentence_transformers import SentenceTransformer
import numpy as np

In [ ]:
import pyodbc

conn1 = pyodbc.connect(
    Driver='{ODBC Driver 18 for SQL Server}',
    Server='<servername>',
    Database='<DB Name>',
    Uid='UID',
    Encrypt='yes',
    TrustServerCertificate='no',
    ConnectionTimeout=30,
    Authentication='ActiveDirectoryIntegrated'
)

# Establish a connection
conn = pyodbc.connect(conn_str)

# Define the SQL query to retrieve 'Title' and 'Plot' columns from dbo.movie_plots
sql_query = 'SELECT Title, Plot FROM dbo.movie_plots'

# Execute the query and read the results into a Pandas DataFrame
imdb_plots_df = pd.read_sql_query(sql_query, conn)

# Close the connection
conn.close()

In [7]:
# Load the pre-trained SentenceTransformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

StatementMeta(, bf117a5a-d958-4c4f-8a46-00bf0a6649aa, 9, Finished, Available, Finished)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

StatementMeta(, bf117a5a-d958-4c4f-8a46-00bf0a6649aa, 11, Finished, Available, Finished)

In [ ]:
# Function to convert text to vector and normalize
def text_to_vector(text):
    vector = model.encode(text, convert_to_numpy=True)
    return vector / np.linalg.norm(vector, axis=1, keepdims=True)

# Convert all movie plots to vectors
plot_vectors = text_to_vector(imdb_plots_df['Plot'].tolist())

# Create a FAISS index
dimension = plot_vectors.shape[1]
index = faiss.IndexFlatIP(dimension)

# Add vectors to the index
index.add(plot_vectors.astype('float32'))

# Save the index to a file
faiss.write_index(index, "/lakehouse/default/Files/movie_plot_miniLM.index")

# Save the DataFrame to a CSV file for later use
imdb_plots_df.to_csv("/lakehouse/default/" + "Files/movie_plots.csv", index=False)